## Simple code explanation

Bien penser à utiliser: 
pip install -r requirements.txt

Installez également graphviz (pour la visualisation du classifier)

Just imports, nothing noticable yet.

In [77]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
from sklearn.feature_extraction.text import CountVectorizer # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.tree import DecisionTreeClassifier # type: ignore
import re
from sklearn.tree import export_graphviz # type: ignore
import graphviz # type: ignore
from sklearn import tree # type: ignore

We are using a dataset created by some MIT students apparently: https://huggingface.co/datasets/christophsonntag/OLID?library=true

This dataset uses tweets as labels and have 3 differents output (I'll let you check it out)

In [78]:
from datasets import load_dataset # type: ignore
dataset = load_dataset("christophsonntag/OLID")

We set x as our data

In [79]:
x = np.array(dataset['train']['cleaned_tweet'])

We set y as our label (expected output)

In [80]:
y = np.array(dataset['train']['subtask_a'])

Just some tricky thingy so the array don't broke.

In [81]:
indices = [i for i, val in enumerate(x) if val is not None]

Adding a filter on val so we don't get junky chinese or numbers. This is a minus for the numbers because I could type R3t0rd and it won't be detected but the dataset isn't build to detect that anyway.
I could have used a method inside the CountVectorizer tho but for testing purpose I did the code bellow.
https://medium.com/swlh/understanding-count-vectorizer-5dd71530c1b 

In [82]:
"""x = [re.sub(r'[^a-zA-Z\s]','',val) for i, val in enumerate(x) if i in indices]
y = [re.sub(r'[^a-zA-Z\s]','',val) for i, val in enumerate(y) if i in indices]"""

Dans le document au dessus concernant notamment les filtres du CountVectorizer, nous avons l'impression d'après nos résultats de confusion matrix que rajouter des filtres fais baisser les résultats.
Peut-être parce que le filtre retire des tests qui devraient être bon si on les laissait ?
L'accuracy est plus grande sans notre filtre, donc nous laissons le code sans filtre pour l'instant, pas de manipulation supplémentaire.

In [ ]:
x = [val for i, val in enumerate(x) if i in indices]
y = [val for i, val in enumerate(y) if i in indices]

Initialisation for the  CountVectorizer

In [83]:
cv = CountVectorizer()
x = cv.fit_transform(x)
cv.get_feature_names_out()


array(['aa', 'aaa', 'aaaahh', ..., 'zoom', 'zoomed', 'zuckerberg'],
      dtype=object)

On vérifie que le filtre appliqué plus haut a bien fonctionné

In [84]:
cv.get_feature_names_out()

array(['aa', 'aaa', 'aaaahh', ..., 'zoom', 'zoomed', 'zuckerberg'],
      dtype=object)

test_size determinize the split value between tests and training

We need to split to calculate the differents metrics like accuracy, confusion matrix, recall etc..

In [85]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

Initisalisation of the DecisionTreeClassifier

Why a DecisionTreeClassifier ? Simply because it is a simple Classifier

In [86]:
clf = DecisionTreeClassifier()

Training the Classifier

In [104]:
clf.fit(x_train, y_train)

DecisionTreeClassifier()

We want to visualize our DecisionTreeClassifier this one take way to long to load (Only activate if you have time lol)

In [105]:
"""from matplotlib import pyplot as plt
from sklearn import tree

feature_names = cv.get_feature_names_out()
class_names = list(set(y_train))  

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, 
                   feature_names=feature_names,  
                   class_names=class_names,
                   filled=True)
plt.show()
"""

'from matplotlib import pyplot as plt\nfrom sklearn import tree\n\nfeature_names = cv.get_feature_names_out()\nclass_names = list(set(y_train))  \n\nfig = plt.figure(figsize=(25,20))\n_ = tree.plot_tree(clf, \n                   feature_names=feature_names,  \n                   class_names=class_names,\n                   filled=True)\nplt.show()\n'

Just some testing on the predict (NOT = "Good phrase" | OFF = "Bad phrase")

In [106]:
test_data = "Hello man!"
df = cv.transform([test_data]).toarray()
test_predict = clf.predict(df)
print(test_predict)

['NOT']


Metrics

In [107]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

y_predict = clf.predict(x_test)

accuracy  = accuracy_score(y_test, y_predict)

print(f'Accuracy: {accuracy:.2f}')

# Calcul de la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_predict)
print('Confusion Matrix:')
print(conf_matrix)

# Calcul de la précision
precision = precision_score(y_test, y_predict, average='weighted')
print(f'Precision: {precision:.2f}')

# Calcul du recall
recall = recall_score(y_test, y_predict, average='weighted')
print(f'Recall: {recall:.2f}')

Accuracy: 0.73
Confusion Matrix:
[[1484  297]
 [ 427  426]]
Precision: 0.72
Recall: 0.73
